<a href="https://colab.research.google.com/github/hudada369/fy2020-repo-config/blob/master/Pytorch07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **优化器**
损失函数的作用是衡量模型输出与真实标签的差异，当我们有了这个 loss 之后，我们就可以通过反向传播机制得到参数的梯度，那么我们如何利用这个梯度进行更新参数使得模型的 loss 逐渐的降低

1. 优化器（什么是优化器，优化器运行机制、属性和方法，常用的优化器介绍）
  1. 优化器：管理并更新模型中可学习参数的值，使得模型输出更接近真实标签。
  2. 梯度是一个向量，它的方向是导数取得最大值的方向，也就是增长最快的方向
  3. 采用梯度下降（沿着梯度的负方向去变化）的方式去更新权值，使得函数的下降尽量的快
2. 学习率调整策略(为什么调整学习率， 六种学习率调整策略）
3. 梳理总结

In [ ]:
class Optimizer(Object):
  def __init__(self,params,defaults):
    self.defaults = defaults
    self.state = defaultdict(dict)
    self.param_groups=[]

    param_groups = [{'params':param_groups}]

In [ ]:
class Optimizer(object):
  def zero_grad(self):
    for group in self.param_groups:
      for p in group["params"]:
        if p.grad is not none:
          p.grad_datach_()
          p.grad_zero_()

**Optimizer 的基本属性和方法**
1. defaults: 优化器超参数，里面会存储一些学习了，momentum 的值，衰减系数等
2. state: 参数的缓存，如 momentum 的缓存（使用前几次梯度进行平均）
3. param_groups: 管理的参数组，这是个列表，每一个元素是一个字典，在字典中有 key，key 里面的值才是我们真正的参数（「这个很重要，进行参数管理」）
4. _step_count: 记录更新次数，学习率调整中使用，比如迭代 100 次之后更新学习率的时候，就得记录这里的 100
**优化器里面的基本方法**
1. zero_grad()：清空所管理参数的梯度， 里注意Pytorch有一个特性就是「张量梯度不自动清零」
2. step(): 执行一步更新
3. add_param_group(): 添加参数组，我们知道优化器管理很多参数，这些参数是可以分组的，我们对不同组的参数可以设置不同的超参数，比如模型 finetune 中，我们希望前面特征提取的那些层学习率小一些，而后面我们新加的层学习率大一些更新快一点，就可以用这个方法

In [ ]:
class Optimizer(object):
  def add_param_group(self.param_group):
    for group in self.param_groups:
      param_set_updata(set(group["params"]))
    self.param_groups.append(param_group)      

4. state_dict(): 获取优化器当前状态信息字典
5. load_state_dict(): 加载状态信息字典，这两个方法用于模型断点的一个续训练， 所以我们在模型训练的时候，一般多少个 epoch 之后就要保存当前的状态信息。


In [ ]:
class Optimizer(object):
  def state_dict(self):
    return {
        'state':packed_state,
        'param_groups':param_groups,
    }
  def load_state_dict(self,state_dict):

1. 初始化 SGD 的时候传入了一个形参：optim.SGD(net.parameters(), lr=LR, momentum=0.9)，这里的net.parameters() 就是神经网络的每层的参数，SGD 在初始化的时候， 会把这些参数以参数组的方式再存起来，上图中的 params 就是神经网络每一层的参数

2. 初始化完了之后， 我们就可以进行梯度清空，然后更新梯度即可
optimizer.zero_grad(),中间是计算损失，反向传播过程，optimizer.step()梯度更新（w = w - lr*w.grad）
3. add_param_group(): 添加参数组 这个是在模型的迁移学习中非常实用的一个方法，我们看看怎么用：
  1. param_groups里面的weight和weight的地址是一样的，存的是引用，这就提到了叶子结点不能进行原位操作
4. state_dict()和load_state_dict() 这两个方法用于保存和加载优化器的一个状态信息，通常用在断点的续训练， 比如我们训练一个模型，训练了10次停电了， 那么再来电的时候我们就得需要从头开始训练，但是如果有了这两个方法，我们就可以再训练的时候接着上次的次数继续， 所以这两个也非常实用。 
  1. state_dict() 方法里面保存了我们优化器的各种状态信息（包括param_groups参数组4），我们通过 torch.save 就可以保存这些状态到文件(.pkl)
  2. load_state_dict() 来导入这个状态信息，让优化器在这个基础上进行训练

# **优化器的运行机制**
管理和更新模型的可学习参数（管理是通过各种属性，尤其是 param_groups 这个重要的属性，而更新是通过各种方法，主要是 step()方法进行更新）

常见优化器：
概念：
1. 学习率：学习率起到了控制参数更新的一个步伐的作用
2. 动量：Momentum：结合当前梯度与上一次更新信息（利用指数加权的方法，对前面步的梯度进行权值相乘，然后用来更新本次梯度）， 用于当前更新
  1. 指数加权平均， 指数加权平均在时间序列中经常用于求取平均值的一个方法，它的思想是这样，我们要求取当前时刻的平均值，距离当前时刻越近的那些参数值，它的参考性越大，所占的权重就越大

优化器：
1. optim.SGD（param，lr,momentum,weight_decay,nesterov）
 1. param: 管理的参数组
 2. lr: 初识学习率
 3. momentum：动量系数， beta
 4. weight_decay: L2 正则化系数
 5. nesterov: 是否采用 NAG
2. optim.SGD: 随机梯度下降法
3. optim.Adagrad: 自适应学习率梯度下降法
4. optim.RMSprop: Adagrad 的改进
5. optim.Adadelta: Adagrad 的改进
6. optim.Adam: RMSprop 结合 Momentum
7. optim.Adamax: Adam 增加学习率上限
8. optim.SparseAdam: 稀疏版的 Adam
9. optim.ASGD: 随机平均梯度下降
10. optim.Rprop: 弹性反向传播
11. optim.LBFGS: BFGS 的改进

学习率调整策略：（先用较快的速率达到最优点附近，然后再将学习率降下来，缓慢的去瘦收敛到最优值）

使用方法：
1. 定义 scheduler = torch.optim.lr_scheduler.StepLR(参数)
2. 使用scheduler.step() 不当到iteration中

基类：
属性：
1. optimizer: 关联的优化器， 得需要先关联一个优化器，然后再去改动学习率
2. last_epoch: 记录epoch数， 学习率调整以epoch为周期
3. base_lrs: 记录初始学习率
方法：
1. step(): 更新下一个epoch的学习率， 这个是和用户对接
2. get_lr(): 虚函数， 计算下一个epoch的学习率， 这是更新过程中的一个步骤

In [ ]:
class _LRScheduler(object):
  def __init__(self,optimizer,last_epoch=-1):
  
  def get_lr(self):
    # 调用的时候会具体实现 我们改变学习率的具体方法
    reiase NotImplementedError

1. 首先我们在定义优化器的时候，这时候会完成优化器的初始化工作， 主要有关联优化器(self.optimizer属性), 然后初始化last_epoch和base_lrs(记录原始的学习率，后面get_lr方法会用到)。
2. 然后就是用Scheduler，我们是直接用的step()方法进行更新下一个epoch的学习率（这个千万要注意放到epoch的for循环里面而不要放到batch的循环里面 ），而这个内部是在_Scheduler类的step()方法里面调用了get_lr()方法， 
3. 而这个方法需要我们写Scheduler的时候自己覆盖，告诉程序按照什么样的方式去更新学习率，这样程序根据方式去计算出下一个epoch的学习率，然后直接更新进优化器的_param_groups()里面去。

学习率六种调整策略
1. StepLR 功能：等间隔调整学习率
  1. lr_scheduler.StepLR(optimizer,step_size,gamma=0.1,last_epoch=-1)
  2. step_size表示调整间隔数,间隔多少epoch调整
  3.  gamma表示调整系数
  4. 调整方式就是lr=lr*gamma, 这里的gamma一般是0.1-0.5
2. MultiStepLR 功能：按给定间隔调整学习率
  1. lr_scheduler.MultiStepLR(optimizer,milestones,gamma=0.1,last_epoch=-1)
  2. milestones表示设定调整时刻数 构建一个list，比如[50, 125, 150]， 放入到milestones中，那么就是50个epoch，125个epoch，150个epoch调整一次学习率
3. ExponentialLR 功能：按指数衰减调整学习率
  1. ExponentialLR（optimizer,gamma,last_epoch=-1）
  2. gamma表示指数的底了
  3. lr = lr *gamma* epoch
4. CosineAnnealingLR 功能：余弦周期调整学习率
  1. CosineAnnealingLR(optimizer,T_max,eta_min=0,last_epoch=-1)
  2. T_max表示下降周期，只是往下的那一块。eta_min表示学习率下限
  3. 
5. ReduceLRonPlateau 功能: 监控指标， 当指标不再变化则调整
  1.  「这个非常实用」。可以监控loss或者准确率，当不在变化的时候，我们再去调整
  2. 参数较多，自学
6. LambdaLR 功能：自定义调整策略，这个也比较实用，可以自定义我们的学习率更新策略，这个就是真的告诉程序我们想怎么改变学习率了。并且「还可以**对不同的参数组设置不同的学习率调整方法**，所以在模型的finetune中非常实用」
  1. LambdaLR（optimizer，lr_lambda,last_epoch=-1）
  2. lr_lambda表示function或者是list

1. 有序调整：Step、MultiStep、 Exponential和CosineAnnealing， 这些得事先知道学习率大体需要在多少个epoch之后调整的时候用
2. 自适应调整：ReduceLROnPleateau， 这个非常实用，可以监控某个参数，根据参数的变化情况自适应调整
3. 自定义调整：Lambda， 这个在模型的迁移中或者多个参数组不同学习策略的时候实用

学习率的初始化方式：
1. 设置较小数：0.01， 0.001， 0.0001
2. 先让学习率从0开始慢慢的增大，然后观察acc， 看看啥时候训练准确率开始下降了，就把初始学习率定为那个数